In [41]:
import json
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.chat.util import Chat, reflections

In [42]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
# Load intents from JSON file
with open('/content/SRU_DATASET.json', 'r') as file:
    intents = json.load(file)

In [44]:
# Tokenization and lemmatization
lemmatizer = nltk.stem.WordNetLemmatizer()
def tokenize_and_lemmatize(text):
    return [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(text)]

In [45]:
# Preprocess data
training_sentences = []
training_labels = []
response_tags = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    response_tags.append(intent['tag'])

In [46]:
# TF-IDF vectorization
vectorizer = TfidfVectorizer(tokenizer=tokenize_and_lemmatize, stop_words='english')
X_train = vectorizer.fit_transform(training_sentences)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [47]:
# Consider exploring alternative ML models for intent classification
# For example, from sklearn: SVC (Support Vector Classification)
# Or consider libraries like spaCy for advanced NLP tasks
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, training_labels)

LogisticRegression(max_iter=1000)

In [48]:
# Conversation state management (optional)
context = {}  # Dictionary to store conversation history

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, training_labels, test_size=0.2, random_state=42)

# Train the classifier on the training set
classifier.fit(X_train, y_train)

# Predict intents for the testing set
y_pred = classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7746478873239436


In [49]:
# Conversation state management (optional)
context = {}  # Dictionary to store conversation history

def get_response(tag, intents_json, context=None):
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            if context and 'context_set' in i:
                # Check for context-specific responses
                if context.get(i['context_set'], False):
                    return random.choice(i['responses']['context_aware'])
            return random.choice(i['responses'])

def chatbot_response(text):
    global context  # Access and update conversational context

    text_vectorized = vectorizer.transform([text])
    predicted_intent = classifier.predict(text_vectorized)[0]

    # Update context based on the predicted intent (optional)
    if predicted_intent in [intent['tag'] for intent in intents['intents'] if 'context_set' in intent]:
        context[predicted_intent] = True  # Flag for context-aware responses

    response = get_response(predicted_intent, intents, context)  # Pass context if applicable
    return response

In [50]:
# Chatbot initialization with reflections (optional)
pairs = [
    ('(.*)', 'Do you want to talk about %1?'),
]
chatbot = Chat(pairs, reflections)

print("Bot: Hello! I am the ChatBot. How can I assist you today? (type 'exit' to end the conversation)")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Bot: Goodbye! Have a great day.")
        break

    # Utilize the Chat library if integrated (optional)
    # response = chatbot.respond(user_input)
    # print("Bot:", response)

    response = chatbot_response(user_input)
    print("Bot:", response)

Bot: Hello! I am the ChatBot. How can I assist you today? (type 'exit' to end the conversation)
You: hi
Bot: Hello!
You: about university
Bot: Our university encourages all-round development of students and hence provides sports facilities in the campus. For more details visit https://sru.edu.in/Campuslife
You: nirf ranking
Bot: NIRF ranking of our university : 98(according to 2024)
You: what should I call you
Bot: Come back soon
You: timing of college
Bot: College is open 9am-5pm Monday-Friday!
You: how to contact college
Bot: You can contact at: +91 8371004040
You: courses offered
Bot: Our university offers IT, CSE,ECE ,EEE, Civil, MBA, BBA and PHD Scholars with specilizations.
You: how much is the fees
Bot: For Fee detail visit this link https://sru.edu.in/fee-and-scholarship-2024
You: where is college located
Bot: The college is located at https://www.google.com/maps/dir/18.0786172,79.4591372/sr+university+google+maps/@18.0852243,79.4565307,15z/data=!3m1!4b1!4m9!4m8!1m1!4e1!1m5!1m1